In [1]:
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from tqdm import tqdm
import sys

_ = load_dotenv(find_dotenv())  # read local .env file
sys.path.append(Path("..").resolve().as_posix())

In [2]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

from src.core.index import Index, ID_KEY
from src.core.retriever import build_retriever
from src.core.generator import build_generator

In [3]:
# index = Index.from_metadata(
#     metadata_path=Path("../resources/doc_metadata.jsonl").resolve(),
#     lib_path=Path("../../docs_md").resolve(),
#     persist_path=Path("index_storage").resolve(),
#     embedding_function=AzureOpenAIEmbeddings(
#         azure_deployment="text-embedding-ada-002", api_version="2023-07-01-preview"
#     ),
# )

index = Index.from_persist_path(
    persist_path=Path("index_storage").resolve(),
    embedding_function=AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002", api_version="2023-07-01-preview"
    ),
)

In [4]:
llm = AzureChatOpenAI(
    temperature=0.0,
    azure_deployment="gpt4o",
    openai_api_version="2023-07-01-preview",
    max_retries=0,
)

In [5]:
retriever = build_retriever(
    llm=llm, vectorstore=index.vectorstore, docstore=index.docstore
)

In [7]:
generator = build_generator(llm=llm, retriever=retriever)

In [8]:
from langchain.globals import set_debug

set_debug(False)

response = generator.invoke(
    {"input": "I have a type with a .created_at and a .last_updated_at both datetime. I want to find the difference in full days between the two. What is the best way to do this in edgeql?"},
    config={
        "configurable": {"session_id": "0"}
    },  # constructs a key "abc123" in `store`.
)

Parent run 62cf2026-5d8c-425c-8d22-c86ba6367ae6 not found for run bda13dac-c925-4d71-a656-08ccb947a3f1. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError()


In [18]:
def pretty_print_response(response):
    print(f"************PROMPT************\n\n{response['input']}\n\n")
    print(f"************ANSWER************\n\n{response['answer'].answer}\n\n")

    print(f"***********CITATIONS**********\n\n")

    for citation in response["answer"].citations:
        doc = response["retrieval_result"]["documents"][citation.source_id]
        print(f"Source {citation.source_id}: {doc.metadata['source']}\n")
        print(f"Category: {doc.metadata['category']}\n")
        print(f"Quote:\n{citation.quote}\n\n")

    print(f"*********SEARCH TERMS*********\n")
    search_terms = response["retrieval_result"]["search_terms"]
    print(f"Query: {search_terms.query}")
    print(f"Filter: {search_terms.category}\n\n")

    print(f"************SOURCES***********\n\n")

    for i, doc in enumerate(response["retrieval_result"]["documents"]):
        print(f"Source {i}: {doc.metadata['source']}\n")
        print(f"Category: {doc.metadata['category']}\n")
        print(f"Content:\n{doc.page_content}\n\n")

In [19]:
pretty_print_response(response)

************PROMPT************

I have a type with a .created_at and a .last_updated_at both datetime. I want to find the difference in full days between the two. What is the best way to do this in edgeql?


************ANSWER************

To find the difference in full days between two datetime values in EdgeQL, you can use the `cal::date_duration` type. This type allows you to represent the difference in days and months between two dates. You can then extract the `days` field from the `cal::date_duration` value to get the difference in full days.


***********CITATIONS**********


Source 2: /Users/andrey/local/projects/edgedb_driven_home/docs_md/reference/protocol/dataformats21.md

Category: edgedb_general

Quote:
[`cal::date_duration`](https://edgedb.com/docs/stdlib/datetime#type::cal::date_duration) values are represented as the following structure:

```c
struct DateDuration {
    int64   reserved;
    int32   days;
    int32   months;
};
```

For example, the `cal::date_duration` 

In [14]:
response

{'input': 'I have a type with a .created_at and a .last_updated_at both datetime. I want to find the difference in full days between the two. What is the best way to do this in edgeql?',
 'chat_history': [],
 'retrieval_result': {'input': 'I have a type with a .created_at and a .last_updated_at both datetime. I want to find the difference in full days between the two. What is the best way to do this in edgeql?',
  'search_terms': Search(query='difference in full days between two datetime fields', category='edgedb_general'),
  'documents': [Document(page_content="# cal::local_date\n\n[`cal::local_date`](https://edgedb.com/docs/stdlib/datetime#type::cal::local_date) values are represented as a 32-bit integer, most sigificant byte first. The value is the number of _days_ between the encoded date and January 1st 2000.\n\nFor example, the `local_date` value `'2019-05-06'` is encoded as:\n\n```c\n0x00 0x00 0x1b 0x99\n```\n\n[Learn more](https://www.edgedb.com/docs/reference/protocol/dataform